# MLP
Test of simple MLP models with different amount of neurons, hidden layers and size of input vector
## Load Data

In [74]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
 
import pandas as pd
import numpy as np

# from tqdm import tqdm

import plotly.express as px
from torch.optim import Adam


import matplotlib.pyplot as plt

In [75]:
df = pd.read_csv("data_N_3000_noise_level_0.05.csv")
df_diff = pd.DataFrame(df.iloc[1:].values - df.iloc[:-1].values, columns=["dt", "dx", "dy", "dz"])

In [76]:
class SequentDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, n_dots=1, n_dot_parameters=4):
        self.n_dot_parameters = n_dot_parameters
        self.n_dots = n_dots
        all_data = self.__make_stack(dataframe)
        self.X_ = all_data[:, :-n_dot_parameters]
        self.y_ = all_data[:, -n_dot_parameters:]
        
    def __len__(self):
        return len(self.X_)
    
    def __getitem__(self, idx):
        return self.X_[idx], self.y_[idx]
    
    def __make_stack(self, df: pd.DataFrame):
        stacks = [df.iloc[:-self.n_dots]] + [df.iloc[i:].values if (self.n_dots == i) else df.iloc[i:-(self.n_dots - i)].values for i in range(1, self.n_dots + 1)]
        return torch.tensor(np.hstack(stacks), dtype=torch.float32)
    
    def plotData(self, i = 0, f = -1):
        fig = px.line_3d(x = self.X_[i:f, 1], y = self.X_[i:f, 2], z = self.X_[i:f, 3])
        fig.show()

In [77]:
N_DOTS = 100
N_DOT_PARAMETERS = 4

In [78]:
dataset = SequentDataset(df_diff[0:2500], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
test = SequentDataset(df_diff[2500:3000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
dataset.plotData()

In [79]:
train_data, val_data = random_split(dataset,[0.8,0.2])

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False)

## Create Model

In [80]:
class MLPModel(nn.Module):
    def __init__(self, input_dots_amount, input_dot_parameters_amount, output_dots_amount, output_dot_parameters_amount, hidden_layers_lengths, dropout_probability=0, activation=nn.ReLU()):
        super(MLPModel, self).__init__()
        self.layers = nn.ModuleList()
        layer_lengths = [input_dots_amount * input_dot_parameters_amount] + hidden_layers_lengths + [output_dots_amount * output_dot_parameters_amount]
        #self.dropout = nn.Dropout(dropout_probability)
        self.activation = activation
        for i in range(1, len(layer_lengths)):
            self.layers.append(nn.Linear(layer_lengths[i - 1], layer_lengths[i]))
    
    
    def forward(self, X: torch.Tensor):
        X = X.clone()
        for layer in self.layers[:-1]:
            X = layer(X)
            X = self.activation(X)
            #X = self.dropout(X)
        return self.layers[-1](X)

In [81]:
model = MLPModel(input_dots_amount=N_DOTS, input_dot_parameters_amount=N_DOT_PARAMETERS, output_dots_amount=1, output_dot_parameters_amount=4, hidden_layers_lengths=[100, 20, 50])
loss_model = nn.MSELoss(reduction='mean')
opt = Adam(model.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=5, gamma=0.95)

In [82]:
model

MLPModel(
  (layers): ModuleList(
    (0): Linear(in_features=40, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=4, bias=True)
  )
  (dropout): Dropout(p=0, inplace=False)
  (activation): ReLU()
)

In [83]:
EPOCH = 250
train_losses = []
val_losses = []

for epoch in range(EPOCH):

    # Обучение
    model.train()
    # train_loop = tqdm(train_loader,leave=False)
    train_loss = []
    for X, y in train_loader:
        pred = model(X)
        loss = loss_model(pred, y)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()

        opt.step()
        mean_train_loss = sum(train_loss)/len(train_loss)
        # train_loop.set_description(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}")

    train_losses.append(mean_train_loss)
    
    # Валидация
    model.eval()
    with torch.no_grad():
        val_loss = []
        for X, y in val_loader:
            pred = model(X)
            loss = loss_model(pred, y)
            val_loss.append(loss.item())

        mean_val_loss = sum(val_loss)/len(val_loss)
        val_losses.append(mean_val_loss)

    lr_scheduler.step()
    lr = lr_scheduler.get_last_lr()
    print(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}, val_loss = {mean_val_loss:.4f}")

  0%|          | 0/63 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x4 and 40x100)

In [67]:
start, _  = dataset[0]
predictions = []
model.eval()

for i in range(1,500):
    with torch.no_grad():
        pred = model(start)
        predictions.append(pred)
        start = torch.hstack([start[N_DOT_PARAMETERS:], pred]) 
        # start = pred   

In [68]:
predictions

[tensor([ 0.9984, -0.0065,  0.0316, -0.0069]),
 tensor([ 9.9818e-01,  3.3896e-04, -1.4709e-03, -8.7937e-03]),
 tensor([ 0.9983, -0.0014,  0.0099, -0.0077]),
 tensor([ 9.9828e-01, -6.4028e-04,  6.0383e-03, -8.0710e-03]),
 tensor([ 9.9828e-01, -8.3076e-04,  7.3550e-03, -7.8624e-03]),
 tensor([ 9.9828e-01, -7.5979e-04,  6.9088e-03, -7.9466e-03]),
 tensor([ 9.9828e-01, -7.6859e-04,  7.0621e-03, -7.9049e-03]),
 tensor([ 9.9828e-01, -7.7073e-04,  7.0097e-03, -7.9240e-03]),
 tensor([ 9.9828e-01, -7.6827e-04,  7.0275e-03, -7.9157e-03]),
 tensor([ 9.9828e-01, -7.6970e-04,  7.0215e-03, -7.9192e-03]),
 tensor([ 9.9828e-01, -7.6902e-04,  7.0235e-03, -7.9178e-03]),
 tensor([ 9.9828e-01, -7.6932e-04,  7.0228e-03, -7.9184e-03]),
 tensor([ 9.9828e-01, -7.6918e-04,  7.0231e-03, -7.9181e-03]),
 tensor([ 9.9828e-01, -7.6924e-04,  7.0230e-03, -7.9182e-03]),
 tensor([ 9.9828e-01, -7.6921e-04,  7.0230e-03, -7.9182e-03]),
 tensor([ 9.9828e-01, -7.6922e-04,  7.0230e-03, -7.9182e-03]),
 tensor([ 9.9828e-01, -7

In [69]:
p_x = pd.DataFrame(map(lambda t: t.detach().tolist(), predictions), columns=["Time", "X", "Y", "Z"])
_, y  = dataset[:500]
p_y = pd.DataFrame(y.detach().numpy(), columns=["Time", "X", "Y", "Z"])

p_x = p_x.cumsum()
p_y = p_y.cumsum()

In [70]:
p_y

,Time,X,Y,Z
0,1.0,-0.048530,0.067327,-0.012345
1,2.0,-0.043451,0.056180,0.027033
2,3.0,0.009593,0.023557,0.030584
3,4.0,-0.000333,0.000965,0.063170
4,5.0,-0.043598,-0.007739,-0.004437
...,...,...,...,...
495,496.0,1.694747,-1.973750,-0.890775
496,497.0,1.636614,-1.954283,-0.961964
497,498.0,1.679461,-1.946530,-0.919627
498,499.0,1.739971,-1.972070,-0.934394


In [71]:
p_x["C"] = "Predict"
p_y["C"] = "True"

d = pd.concat([p_x,p_y],axis=0)

In [72]:
fig = px.line_3d(d, x="X", y="Y", z="Z", color='C')
# fig = px.scatter_3d(data, x="X", y="Y", z="Z")
fig.show()